# Development Notebook
Constanly changing

In [ ]:
import xarray as xr
import pandas as pd
import os
import pyproj
import numpy as np
import xesmf as xe
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

import noaa_csl_funcs as ncf
from regrid_data import RegridInputs
%load_ext autoreload
%autoreload 2

## Unit conversion

Ongoing work about unit conversion. The official NOAA CSL documentation has some information, but there are some assumptions to be made about molecular weights (SAPRC versioning). This is an attempt to create a package that will:
* Choose and standardize the speciation values
* Allow for easy unit conversion, particularly chemical g/mol type conversions
* Correctly change attributes, labels, etc so things don't get wonky   

Ongoing development with noaa_csl_funcs.Unit_Converter

In [11]:
class CSLSpeciesDetails:
    def __init__(self,csv_fullpath):
        self.csv_fullpath = csv_fullpath
        self.df = self.load_details_csv()

    def load_details_csv(self,csv_fullpath = None):
        if csv_fullpath is None:
            csv_fullpath = self.csv_fullpath
        
        species_details_df = pd.read_csv(csv_fullpath,header = 17)
        return species_details_df
    
    def make_mw_dict(self,saprc_version = 'either'):
        mw_dict = {}
        if saprc_version == 'either':
            for _,row in self.df.iterrows():
                species = row['NC_name']
                mw_dict[species] = {}
                if row['Known_MW'] == row['Known_MW']: #known mw is not a nan, so use this first
                    mw_dict[species]['mw(g/mol)']=row['Known_MW']
                    mw_dict[species]['mw_source'] ='Known_MW'
                elif row['SAPRC99_MW'] == row['SAPRC99_MW']: #use saprc99 first
                    mw_dict[species]['mw(g/mol)'] = row['SAPRC99_MW']
                    mw_dict[species]['mw_source'] = 'SAPRC99_MW'        
                elif row['SAPRC22_MW'] == row['SAPRC22_MW']: #use saprc22 next
                    mw_dict[species]['mw(g/mol)'] = row['SAPRC22_MW']
                    mw_dict[species]['mw_source'] = 'SAPRC22_MW'      
                else:
                    mw_dict[species]['mw(g/mol)'] = np.nan
                    mw_dict[species]['mw_source'] = np.nan   
        else:
            raise ValueError('Havent made anything other that "either"')  #could do SAPRC22 only, 99 only, mixed etc TODO
        return mw_dict
    
csd = CSLSpeciesDetails('/uufs/chpc.utah.edu/common/home/u0890904/NOAA_CSL/noaa_csl/NOAA_CSL_species.csv')
csl_mw_dict = csd.make_mw_dict()
csl_mw_dict


{'CO': {'mw(g/mol)': 28.01, 'mw_source': 'Known_MW'},
 'CO2': {'mw(g/mol)': 44.01, 'mw_source': 'Known_MW'},
 'HC01': {'mw(g/mol)': 16.04, 'mw_source': 'Known_MW'},
 'HC02': {'mw(g/mol)': 30.07, 'mw_source': 'SAPRC99_MW'},
 'HC03': {'mw(g/mol)': 36.73, 'mw_source': 'SAPRC99_MW'},
 'HC04': {'mw(g/mol)': 58.61, 'mw_source': 'SAPRC99_MW'},
 'HC05': {'mw(g/mol)': 77.6, 'mw_source': 'SAPRC99_MW'},
 'HC06': {'mw(g/mol)': 118.89, 'mw_source': 'SAPRC99_MW'},
 'HC07': {'mw(g/mol)': 28.05, 'mw_source': 'SAPRC99_MW'},
 'HC08': {'mw(g/mol)': 72.34, 'mw_source': 'SAPRC99_MW'},
 'HC09': {'mw(g/mol)': 75.78, 'mw_source': 'SAPRC99_MW'},
 'HC10': {'mw(g/mol)': 68.12, 'mw_source': 'SAPRC99_MW'},
 'HC11': {'mw(g/mol)': 136.24, 'mw_source': 'SAPRC99_MW'},
 'HC12': {'mw(g/mol)': 95.16, 'mw_source': 'SAPRC99_MW'},
 'HC13': {'mw(g/mol)': 118.72, 'mw_source': 'SAPRC99_MW'},
 'HC14': {'mw(g/mol)': 30.03, 'mw_source': 'SAPRC99_MW'},
 'HC15': {'mw(g/mol)': 44.05, 'mw_source': 'SAPRC99_MW'},
 'HC16': {'mw(g/mol)'

# Other random stuff

In [ ]:
import subprocess
def create_symlink_of_tree(source_fullpath,dest_path):
    '''Creates symlinks for an entire directory tree
    
    Args:
    source_fullpath (str) : full path of the source of symlinked data. should start and end with /
    dest_path (str) : path to where the symlinked copy will go
    '''
    if not source_fullpath.endswith('/'):
        source_fullpath = source_fullpath+'/'
    command = ["cp","-as", source_fullpath,dest_path]
    print(command)
    subprocess.Popen(command)

def symlink_all_pointsectors(base_path,dest_path):
    for sector in os.listdir(base_path):
        if sector.startswith('point'):
            source_fullpath = os.path.join(base_path,sector)
            dest_fullpath = os.path.join(dest_path,sector)
            
            create_symlink_of_tree(source_fullpath,dest_fullpath)

base_path = '/uufs/chpc.utah.edu/common/home/lin-group9/agm/NOAA_CSL_Data/base'
dest_path = '/uufs/chpc.utah.edu/common/home/lin-group9/agm/NOAA_CSL_Data/regridded'
#symlink_all_pointsectors(base_path,dest_path)
